In [ ]:
# import libraries and datasets

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# load the dataset

behavior = pd.read_csv("data/MINDsmall_dev/behaviors.tsv", sep="\t", header=None, names=["Impression ID", "User ID", "Time", "History", "Impressions"])
news = pd.read_csv("data/MINDsmall_dev/news.tsv", sep="\t", header=None, names=["News ID", "Category", "Subcategory", "Title", "Abstract", "URL", "Title Entities", "Abstract Entities", "Title Topics", "Abstract Topics"])
#TODO: read embeddings

In [ ]:
#TODO: find tags of an article
def article_tags(articleID):
    tags = []
    article = news.loc[news["News ID"] == articleID]
    tags.append(article["Category"].values[0])
    tags.append(article["Subcategory"].values[0])
    return tags[0], tags[1]

#TODO: create user vector
def create_x_i(userID):
    x_i = {}

    # Weights
    w_c = 0.3  
    w_s = 1.0

    article_count = 0

    for index, row in behavior.iterrows():
        if row["User ID"] == userID:
            for articleID in row["History"].split(" "):
                article_count += 1

                category, subcategory = article_tags(articleID)

                if category in x_i:
                    x_i[category] += w_c
                else:
                    x_i[category] = w_c
                
                if subcategory in x_i:
                    x_i[subcategory] += w_s
                else:
                    x_i[subcategory] = w_s
    
    for trait, score in x_i.items():
        x_i[trait] = score/article_count

    return x_i

exID = behavior["User ID"].values[1]

print(create_x_i(exID))